In [ ]:
# converts txt to csv files
import csv 

def txt_to_csv(txt_file):
    csv_file = txt_file.split('/')[-1].split('.')[0] + '.csv'
    csv_file = '/'.join(txt_file.split('/')[:-1]) + '/' + csv_file
    
    inp_data = list()
    with open(txt_file, 'r') as in_file:
        stripped = (line.strip() for line in in_file)
        inp_data = [line.split("\t") for line in stripped if line]
    
    # creating dataframe
    df = pd.DataFrame(inp_data[1:-2], columns=inp_data[0])
    print(df.head())
    df.to_csv(csv_file)

In [ ]:
# Loading data
crime_data = pd.read_csv("../../../../Downloads/Crimes_-_2001_to_present.csv")
analysis_df = crime_data

In [3]:
# Analysis harness
import pandas as pd

# To show different metrics
# analysis_df.info()
# analysis_df['Arrest'].describe()

# import numpy as np
# from matplotlib import cm
# color = cm.inferno_r(np.linspace(.4,.8, 30))

# analysis_df.groupby(['Primary Type', 'Arrest']).size().unstack().plot(kind='barh', stacked=True, figsize=(10,10))
# df2 = analysis_df.groupby(['Primary Type', 'Arrest'])['Primary Type'].count().unstack('Arrest').fillna(0)
# print (df2.head())
# df2[[False,True]].plot(kind='bar', stacked=True)

# analysis_df['Primary Type'].value_counts().plot(kind='barh', figsize=(10,10), color=color)
# analysis_df.isnull().sum().plot(kind='barh', figsize=(10,10), color=color)
# analysis_df[analysis_df['Arrest']==False]['Primary Type'].value_counts().plot(kind='pie', figsize=(10,10))
# df = analysis_df[analysis_df['Arrest']==False]['Primary Type'].value_counts()
# python_pie3D(df.data, df.index)

# To get null values
# analysis_df.isnull().sum()

# Finding co-relation
# correlation_matrix = analysis_df.corr().round(2)
# sns.heatmap(data=correlation_matrix, annot=True)

In [5]:
# REST API Client
import requests

class API_client:
    def __init__(self, url, username, password):
        self._url = url
        self._auth = (username, password)
        
    def set_url(self, url):
        '''
        to reset url
        '''
        self._url = url
        
    def get_response(self):
        '''
        get response from url
        '''
        resp = requests.get(self._url, auth=self._auth)
        
        if resp.status_code != 200:
            # This means something went wrong.
            raise ApiError('GET rest API issue: {}'.format(resp.status_code))
            
        return resp.json()
    
    def put_response(self, data):
        '''
        put response
        '''
        resp = requests.post(self._url, auth=self._auth, json=data)
        
        if resp.status_code != 201:
            raise ApiError('POST rest API issue: {}'.format(resp.status_code))
            
        return 'POST successful'
        
    def __str__(self):
        return 'This is a REST API functions wrapper'

In [ ]:
# analysis_df = analysis_df.to_json(orient='index')
host_name = 'http://localhost:8091'
rest_client = API_client(couchbase_url, 'administrator', 'nainyjain')

# Exploration
url_1 = '/pools/default/buckets'

rest_client.set_url(host_name+url_1)
print (rest_client.get_response())

In [1]:
# Loading data
import pandas as pd
crime_data = pd.read_csv("../../../../Downloads/Crimes_-_2001_to_present.csv")

In [6]:
# Data Ingestion

# Global connection objects
import json
import datetime
from couchbase.cluster import Cluster
from couchbase.cluster import PasswordAuthenticator

# Global DB connection objects
cluster = Cluster('couchbase://34.229.250.146')
authenticator = PasswordAuthenticator('admin', 'password')
cluster.authenticate(authenticator)
cb = cluster.open_bucket('crime_data')

# Sequential run
start_time = datetime.datetime.now()

print ('Inserting data')
for index, doc in enumerate(crime_data[:2500].T.to_dict().values()):
    doc_id = 'k' + str(index)
    cb.upsert(doc_id, doc)
    
end_time = datetime.datetime.now()
print ('Sequential Timing: ', (end_time - start_time).total_seconds())

Inserting data
Sequential Timing:  3.682948


In [42]:
# Method to connect to DB and load data
def push_data(data, cb, thread_name):
    '''
    Push data using parallel threads
    '''
    # Looping and inserting data
    for index, doc in enumerate(data):
        doc_id = 'k' + str(index)
        cb.upsert(doc_id, doc)
        
    print ('Thread completed insertion: ', thread_name)

In [43]:
# Client code for data insertion
import threading

threads = 2
chunk_size = 10
jobs = []
start_time = datetime.datetime.now()

for i in range(0, threads):
    data = crime_data[chunk_size*i:chunk_size*(i+1)].T.to_dict().values()
    thread = threading.Thread(target=push_data(data, cb, 'Thread:'+str(i+1)))
    jobs.append(thread)

# Start the threads (i.e. calculate the random number lists)
for j in jobs:
    j.start()

# Ensure all of the threads have finished
for j in jobs:
    j.join()

print ("List processing complete.")
end_time = datetime.datetime.now()
print ('Parallel Threading Time with global connection object: ', (end_time - start_time).total_seconds())

_TimeoutError_0x17 (generated, catch TimeoutError): <Key='k0', RC=0x17[Client-Side timeout exceeded for operation. Inspect network conditions or increase the timeout], Operational Error, Results=1, C Source=(src/multiresult.c,316), Tracing Output={"k0": {"s": "kv:Unknown", "i": 2016389708, "b": "crime_data", "r": "172.17.0.2:11210", "t": 2500000}}>